In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, callbacks
from tensorflow.keras.applications import EfficientNetB3

def get_data_generators(train_dir, img_size=(300, 300), batch_size=32):
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        rotation_range=10,
        validation_split=0.2
    )
    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )
    val_generator = val_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation',
        shuffle=False
    )
    return train_generator, val_generator

def build_model(img_size=(300, 300)):
    base_model = EfficientNetB3(include_top=False, input_shape=(*img_size, 3), weights='imagenet', pooling='avg')
    base_model.trainable = False  # Freeze base initially

    inputs = layers.Input(shape=(*img_size, 3))
    x = base_model(inputs, training=False)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

def train(train_dir, epochs=15, batch_size=32, img_size=(300, 300)):
    train_generator, val_generator = get_data_generators(train_dir, img_size, batch_size)
    model = build_model(img_size)

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    checkpoint = callbacks.ModelCheckpoint('best_soil_model_tf.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
    early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stop]
    )

    # Fine-tune: unfreeze last 30 layers
    base_model = model.layers[1]
    base_model.trainable = True
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss=losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stop]
    )

if __name__ == "__main__":
    train_dir = "path/to/train_dir"  # Change to your dataset directory
    train(train_dir)
